In [ ]:
conda install -c conda-forge huggingface_hub

In [6]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install Pillow

In [1]:
import pandas as pd
import io
from PIL import Image

# Read dataset

In [2]:
# Read the dataset on pandas and join train and test
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/doceoSoftware/docvqa_clicars_facturavenda_Mireia_600_3/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/doceoSoftware/docvqa_clicars_facturavenda_Mireia_600_3/" + splits["test"])

df = df_train._append(df_test, ignore_index=True) 
print(f"Shape of the dataframe: {df.shape}")
df.head()

Shape of the dataframe: (603, 4)


,image,query,answers,ground_truth
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Matricula?, Bastidor?]","[2433LYC, VR3UPHNEKN5827691]","{""gt_parses"": [{""question"" : ""Matricula?"", ""an..."
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Matricula?, Bastidor?]","[4220-LXC, VF72RHNPMN4166622]","{""gt_parses"": [{""question"" : ""Matricula?"", ""an..."
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Matricula?, Bastidor?]","[4222-LXC, VF72RHNPMN4164785]","{""gt_parses"": [{""question"" : ""Matricula?"", ""an..."
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Matricula?, Bastidor?]","[4223-LXC, VF72RHNPMN4166623]","{""gt_parses"": [{""question"" : ""Matricula?"", ""an..."
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Matricula?, Bastidor?]","[4338-LPL, VXKUPHNEKM4146585]","{""gt_parses"": [{""question"" : ""Matricula?"", ""an..."


In [9]:
image_bytes = df["image"][0]["bytes"]
# Create an in-memory file-like object from the image bytes
image_file = io.BytesIO(image_bytes)
print(image_file)
# Open the image using PIL
image = Image.open(image_file)
small_image = image.resize((200, 300))
# Display the image
small_image.show()

# OCR read images

In [17]:
# 
from pydantic import BaseModel, Field
from datetime import datetime
from PIL import Image
import pytesseract
import io
    
text = pytesseract.image_to_string(image, lang='eng')


# Mostrar el texto extraído y los datos de la factura
print("Texto extraído con OCR:")
print(text)

Texto extraído con OCR:
STELLANTIS ESPARA, 5.L.

Son10- ico -Pomeveta INVOICE
moe Used Car

NVIDENTFICATION:ES850629187

[nvotce 3300027907 ‘Order 320000 STE FR ORERE SOVETOT
Seder date 30.08.2023

‘Ship to ages: AVOA MANOTERAS, NUM
3

cucans.

33
28050- MADRID
Span

ncoterm = EXW (deposit init)

<aused Car description,
eave Pevcror aan RP Aare Paretech TOO 365 Va on
ess= ave Neen ennsearens €02 ihn: 125 Wein:

ideas) Esbtodrefrlehmentcots rest) 62145 Seed
Besa price dtais

Freetar charge ___Taxrote Tex omount__Alrtu charae
| ican pein barge ans Tae ae aemnase ——Tssansd|

(Ferme & Condition ]

neceta en Resto Hercati de Pontevedra, Tomo 40, Uo 444, Flo. Sec

oj PO 72.351, erp




# LLM

In [22]:
pip install langchain

     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ---------------------------------------- 51.6/51.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/983.6 kB ? eta -:--:--
   -------------------------------- ------ 819.2/983.6 kB 17.2 MB/s eta 0:00:01
   --------------------------------------- 983.6/983.6 kB 15.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/366.3 kB ? eta -:--:--
   --------------------------------------- 366.3/366.3 kB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/127.9 kB ? eta -:--:--
   ---------------------------------------- 127.9/127.9 kB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/136.4 kB ? eta -:--:--
   ---------------------------------------- 136.4/136.4 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/54.0 kB ? eta -:--:--
   ---------------------------------------- 54.0/54.0 kB 2.9 MB/s eta 0:00:00
  Attempting uni

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


In [24]:
from langchain.output_parsers import PydanticOutputParser

class Facture(BaseModel):
    """Facture Datamodel"""
    date : datetime = Field(default = None, description = "Date of the facture") 
    monto : float = Field(default = None, description = "Identifier of the facture")
    facture_number: str = Field(default = None, description = "")

parser = PydanticOutputParser(pydantic_object = Facture)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Facture Datamodel", "properties": {"date": {"title": "Date", "description": "Date of the facture", "type": "string", "format": "date-time"}, "monto": {"title": "Monto", "description": "Identifier of the facture", "type": "number"}, "facture_number": {"title": "Facture Number", "type": "string"}}}
```


In [39]:
system_template = """
You have to perform the task of extracting information from data.

Extract the data following the format:
{format_instructions}
"""

system_instructions = system_template.format(
    format_instructions=parser.get_format_instructions(),
)
system_instructions

'\nYou have to perform the task of extracting information from data.\n\nExtract the data following the format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Facture Datamodel", "properties": {"date": {"title": "Date", "description": "Date of the facture", "type": "string", "format": "date-time"}, "monto": {"title": "Monto", "description": "Identifier of the facture", "type": "number"}, "facture_number": {"title": "Facture Number", "type": "string"}}}\n```\n'

In [40]:
prompt_template = """
Data:
{data}
"""

prompt = prompt_template.format(
    data=text,
)

print(prompt)


Data:
STELLANTIS ESPARA, 5.L.

Son10- ico -Pomeveta INVOICE
moe Used Car

NVIDENTFICATION:ES850629187

[nvotce 3300027907 ‘Order 320000 STE FR ORERE SOVETOT
Seder date 30.08.2023

‘Ship to ages: AVOA MANOTERAS, NUM
3

cucans.

33
28050- MADRID
Span

ncoterm = EXW (deposit init)

<aused Car description,
eave Pevcror aan RP Aare Paretech TOO 365 Va on
ess= ave Neen ennsearens €02 ihn: 125 Wein:

ideas) Esbtodrefrlehmentcots rest) 62145 Seed
Besa price dtais

Freetar charge ___Taxrote Tex omount__Alrtu charae
| ican pein barge ans Tae ae aemnase ——Tssansd|

(Ferme & Condition ]

neceta en Resto Hercati de Pontevedra, Tomo 40, Uo 444, Flo. Sec

oj PO 72.351, erp





In [44]:
pip install openai

   ---------------------------------------- 0.0/328.5 kB ? eta -:--:--
   --------------------------------------  327.7/328.5 kB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 328.5/328.5 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
   ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [94]:
from openai import OpenAI

OPENAI_API_KEY = "sk-proj-6sS45wurJlK00yJIT5ozT3BlbkFJ0cMWzr3c9SdKBW4ECu7z"


llm_model = "gpt-4o"
llm_client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [95]:
import json
import requests


chat_completion = llm_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_instructions,
        },
        {
            "role":"user",
            "content": prompt,
        },
    ],
    model=llm_model,
)

try:
    response = chat_completion.choices[0].message.content
    info: UserDetail = parser.parse(response)
except Exception as e:
    raise Exception(f"Error parsing response: {e}")

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [16]:
# result["amazon"]


# files = {'file': ('image.png', df["image"][13]["bytes"])}

In [8]:
from typing import List
from pydantic import BaseModel
import pytesseract
from PIL import Image

class ImageOCR(BaseModel):
    image_path: str  # Path to your image file

def perform_ocr(image_data: ImageOCR) -> str:
    try:
        image = Image.open(image_data.image_path)
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        return f"Error processing image: {str(e)}"

# Example usage
image_data = ImageOCR(image_path="path/to/your/image.png")
extracted_text = perform_ocr(image_data)
print(extracted_text)

ModuleNotFoundError: No module named 'pytesseract'